In [ ]:
import requests
import os
from datetime import datetime, timedelta
import pandas as pd
import json


# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")


# 設定
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:


def get_weather_forecast(api_key, lat, lon):
    """
    緯度経度を指定してOpenWeatherMap APIから天気予報を取得する
    arges:
        api_key: OpenWeatherMap APIのキー
        lat: 緯度
        lon: 経度
    return:
        天気データのリスト
    """
    url = "https://api.openweathermap.org/data/2.5/forecast"
    
    weather_data = []
    params = {
            'lat': lat,
            'lon': lon,
            'appid': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data.extend(data.get('list', []))
    else:
        print(response)
    
    return weather_data




In [ ]:
# get_weather_from_cityから仙台市の天気予報を取得
lat = 38.2682
lon = 140.8694
api_key = os.environ["WEATHER_API_KEY"]

jsondata = get_weather_forecast(api_key, lat, lon)
jsondata

In [ ]:
print("天気：",jsondata[0]["weather"][0]["main"])
print("天気詳細：",jsondata[0]["weather"][0]["description"])

print("気温：",jsondata[0]["main"]["temp"])
print("最高気温：",jsondata[0]["main"]["temp_max"])
print("最低気温：",jsondata[0]["main"]["temp_min"])
print("体感気温：",jsondata[0]["main"]["feels_like"])
print("最低気温：",jsondata[0]["main"]["temp_min"])
print("最高気温：",jsondata[0]["main"]["temp_max"])
print("気圧：",jsondata[0]["main"]["grnd_level"])
print("湿度：",jsondata[0]["main"]["humidity"])
print("降水量：",jsondata[0]["rain"]["3h"])

print("風速：",jsondata[0]["wind"]["speed"])
print("風の方角：",jsondata[0]["wind"]["deg"])
print("雲量：",jsondata[0]["clouds"]["all"])
print("降水確率：",jsondata[0]["pop"])
print("日時：",jsondata[0]["dt_txt"])

In [ ]:
#jsondataを保存
with open('data/weatherdata.json', 'w') as f:
    json.dump(jsondata, f, indent=4)

In [ ]:
def get_weather_history(api_key, lat, lon):
    """
    緯度経度から天気データを取得する
    arges:
        api_key: OpenWeatherMapのAPIキー
        lat: 緯度
        lon: 経度
    return:
        天気データのリスト

    """
    url = "https://api.openweathermap.org/data/2.5/onecall/timemachine"
    
    weather_data = []
    params = {
            'lat': lat,
            'lon': lon,
            'appid': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data.extend(data.get('list', []))
    else:
        print(response)
        print(f"Error fetching data for {start_date.strftime('%Y-%m-%d')}: {response.status_code}")
    
    return weather_data

In [ ]:
df = pd.read_csv('data/input/point_history_cleansing_2.csv')

In [ ]:
# dfのuse_dateがに並び替え
df = df.sort_values('use_date')
df[:10]

In [ ]:
# '市'または'群'に続く文字を削除
df['municipality'] = df['municipality'].str.replace(r'(市|郡).*', r'\1', regex=True)
unique_municipalities = df['municipality'].unique()
unique_municipalities

In [ ]:
# 各市町村の最初の利用日と最後の利用日を取得
date_range_per_municipality = df.groupby('municipality')['use_date'].agg(['min', 'max'])
date_range_per_municipality

In [ ]:
df_2 = pd.read_csv('data/input/仙台.csv', encoding='cp932')[2:].reset_index(drop=True)

In [ ]:
df_2[:10]